# HTML解析入门及准备URL生成连续技
![for humans](https://requests-html.kennethreitz.org/_static/requests-html-logo.png#thumbnail)

*  本周主要内容：HTML解析（parse HTML）及准备URL生成连续技
*  上周主要内容：HTML解析（parse HTML）及Xpath实践
*  20春_Web数据挖掘_week03
*  电子讲义设计者：廖汉腾, 许智超
<br/>
<br/>

-----
## 复习

复习：上周内容，实践

* HTML解析（parse HTML）: requests-html  丶
* Xpath实践
* m.liepin.com 取工作牛肉

-----
## 本周内容及学习目标

本周内容聚焦在

<mark> 如何从一页开始有系统的找更多页的内容 </mark>

我们除了继续学习解决上一周已开始面对的以下挑战：
![Xpath Axis](http://krum.rz.uni-mannheim.de/inet-2005/images/xpath-axis.gif)

### 旧目标
1. 使用 requests-html 爬取并存取网页文字档，查找[requests-html 中文文档](https://cncert.github.io/requests-html-doc-cn/#/)
2. 熟悉 [xpath 语法](https://www.w3cschool.cn/xpath/xpath-syntax.html)丶[xpath 节点](https://www.w3cschool.cn/xpath/xpath-nodes.html)
3. 使用 [xpath cheatsheet](https://devhints.io/xpath)
  * 在 Chrome Inspector 使用
  * 在 requests-html (Python) 使用
4. 简易使用 [pd.DataFrame](https://www.pypandas.cn/doc/getting_started/dsintro.html#dataframe)

### 新目标
这一周，学生将实践
* 猎聘PC版 liepin.com 取工作URL参数的牛肉
* 如何生成一连串新URL以进一步爬取数据




In [12]:
%%html
<style>
/* 本电子讲义使用之CSS */
div.code_cell {
    background-color: #e5f1fe;
}
div.cell.selected {
    background-color: #effee2;
    font-size: 2rem;
    line-height: 2.4rem;
}
div.cell.selected .rendered_html table {
    font-size: 2rem !important;
    line-height: 2.4rem !important;
}
.rendered_html pre code {
    background-color: #C4E4ff;   
    padding: 2px 25px;
}
.rendered_html pre {
    background-color: #99c9ff;
}
div.code_cell .CodeMirror {
    font-size: 2rem !important;
    line-height: 2.4rem !important;
}
.rendered_html img, .rendered_html svg {
    max-width: 60%;
    height: auto;
    float: right;
}

.rendered_html img[src*="#full"], .rendered_html svg[src*="#full"] {
    max-width: 100%;
    height: auto;
    float: none;
}

.rendered_html img[src*="#thumbnail"], .rendered_html svg[src*="#thumbnail"] {
    max-width: 15%;
    height: auto;
}

/* Gradient transparent - color - transparent */
hr {
    border: 0;
    border-bottom: 1px dashed #ccc;
}
.emoticon{
    font-size: 5rem;
    line-height: 4.4rem;
    text-align: center;
    vertical-align: middle;
}
.bg-split_apply_comine {
    width: 500px;     
    height: 300px;
    background: url('02_split-apply-comine_500x300.png') -10px -10px;
    float: right;
}
.bg-comine {
    width: 175px;
    height: 150px;
    background: url('02_split-apply-comine_500x300.png') -280px -80px;
    float: right;
}
.bg-apply {
    width: 155px;
    height: 225px;
    background: url('02_split-apply-comine_500x300.png') -160px -30px;
    float: right;
}
.bg-split {
    width: 205px;
    height: 225px;
    background: url('02_split-apply-comine_500x300.png') -10px -30px;
    float: right;
}
.break {
                   page-break-after: right; 
                   width:700px;
                   clear:both;
}
</style>

In [13]:
# 基本模块
import pandas as pd
from requests_html import HTMLSession

# 0. 上周加分作业解答

In [14]:
# C-1   单一页面
url = "https://m.liepin.com/zhaopin/?keyword=PRD"
session = HTMLSession()
r = session.get( url )

# C-5
# 难: '公司URL', '时间', '经验'

# 先取特定元素, 精准打击其子后辈
主要元素 = r.html.xpath( \
    '//div[@class="job-card-wrap"]//div[@class="job-card"]')

# 作为xpath字典，键为我要抓的牛肉名称，值为xpath
dict_xpaths={ 
    'text': {
        '经验':      './/ul/li[time]/text()'
    },
    'text_content': {
        '职称':    './/ul/li/a[contains(@class,"job-name")]/span[@class="name-text"]', 
        '薪水':    './/ul/li/a[contains(@class,"job-name")]/following-sibling::span', 
        '公司地点':'.//ul/li/time/following-sibling::a',
        '公司名称': './/ul/li/a[contains(@class,"company-name")]', 
        '时间':    './/ul/li/time', 
    },
    'href': {
        '链结':    './/ul/li/a[contains(@class,"job-name")]', 
        '公司URL': './/ul/li/a[contains(@class,"company-name")]', 
    }
}

def get_e_text_content(_xpath_):
    # 高级列表推导
    暂存结果 = [e.xpath(_xpath_)[0].lxml.text_content() for e in 主要元素]
    return(暂存结果)

def get_e_text(_xpath_):
    # 高级列表推导
    暂存结果 = ["".join([x.strip() for x in e.xpath(_xpath_)]) for e in 主要元素]
    return(暂存结果)

def get_e_href(_xpath_):
    # 高级列表推导
    暂存结果 = [list(e.xpath(_xpath_, first=True).absolute_links)[0] \
               if len(e.xpath(_xpath_, first=True).absolute_links) >= 1  \
               else "" for e in 主要元素]
    return(暂存结果)

# 只对主要元素下进行.xpath取值
数据字典 = dict()

数据字典 = {k:get_e_text_content(v) for k,v in dict_xpaths['text_content'].items()}
数据字典.update({k:get_e_text(v) for k,v in dict_xpaths['text'].items()})
数据字典.update({k:get_e_href(v) for k,v in dict_xpaths['href'].items()})

[len(v) for k,v in 数据字典.items()]

数据 = pd.DataFrame(数据字典)
数据.to_excel("20春_Web数据挖掘_week02_liepin.xlsx", sheet_name="搜查结果")
数据 

职称          薪水      公司地点                公司名称  \
0                         旅游产品经理   12-20k·12薪        广州      前海爱讯科技(深圳)有限公司   
1                    教育科技 软件产品经理   12-18k·12薪        广州            融捷投资控股集团   
2                           产品经理   12-18k·12薪    广州-海珠区       广州大白互联网科技有限公司   
3                           实施经理   16-23k·12薪     广州-大沙     广东卓志供应链服务集团有限公司   
4                        互联网产品经理   10-15k·12薪     广州-琶洲       广东车海洋环保科技有限公司   
5                         后台产品经理   10-20k·12薪        广州       广东南方新媒体股份有限公司   
6                        区块链产品经理   15-25k·12薪    广州-黄埔区    北京普瑞未来教育科技集团有限公司   
7                         高级产品经理   20-25k·13薪        广州             某软件开发企业   
8                     产品经理（电商系统）   25-40k·14薪  广东,深圳,广州            知名跨境电商公司   
9                        WMS产品经理   20-35k·14薪        广州           某知名跨境电商平台   
10                   产品经理（支付/后端）   10-20k·12薪    广州-海珠区       北京路客互联网科技有限公司   
11                          产品总监   50-70k·13薪        广州                名创优品   
12                          产品专员     5-8k·12薪    广州-海珠区       广州三易互联网科技有限公司   
13                          产品助理     5-8k·13薪    广州-海珠区       广州三易互联网科技有限公司   
14                          产品经理   10-20k·12薪    广州-天河区      广州易达建信科技开发有限公司   
15                          产品经理   14-22k·12薪        广州      锦江信息技术(广州)有限公司   
16                       供应链产品经理   10-23k·12薪    广州-黄埔区                 健客网   
17                     商品经理/产品经理   12-18k·12薪        广州        广州苑同电子商务有限公司   
18                          产品经理   15-25k·12薪    广州-海珠区        青木数字技术股份有限公司   
19                          商品经理   12-18k·12薪   广州-珠江新城        广州苑同电子商务有限公司   
20                    产品经理（校园招聘）    8-12k·13薪        广州              佳都新太科技   
21                          项目经理   12-16k·12薪    广州-番禺区          广州鲸睿科技有限公司   
22  WXG03-微信公众号小程序生活服务行业产品经理（广州）           面议        广州                  腾讯   
23                   大数据产品经理（高级）   10-20k·12薪     广州-元岗          广州丰石科技有限公司   
24             数据技术架构师（南京/广州/西安）   18-30k·12薪        广州        南京海翊数据技术有限公司   
25               高级业务架构师（数字化新零售）           面议    广州-海珠区          广州滴普科技有限公司   
26                  产品经理（智能终端产品）           面议        广州              佳都新太科技   
27                      产品经理（营收）   20-30k·12薪        广州    上海翡翠东方网络信息技术有限公司   
28           融捷健康智能电子公司-智能硬件产品经理           面议        广州            融捷投资控股集团   
29                   产品经理-内容优化方向   15-25k·12薪        广州    上海翡翠东方网络信息技术有限公司   
30                     平台SDK产品经理   15-25k·12薪        广州    上海翡翠东方网络信息技术有限公司   
31                        账号产品经理   15-25k·12薪        广州    上海翡翠东方网络信息技术有限公司   
32                        商家产品经理   25-50k·12薪        广州             Fordeal   
33                  资深产品经理（相机产品）           面议        广州                网易集团   
34                          产品经理   20-40k·12薪        广州                卓尔人人   
35                  产品经理（用户体验改善）   15-25k·12薪    广州-天河区                嘟比英语   
36                          产品经理   15-20k·13薪        广州        广州诚迈信息科技有限公司   
37                        高级产品经理   25-35k·15薪    广州-天河区    上海翡翠东方网络信息技术有限公司   
38                          产品专员    6-10k·15薪    广州-天河区    上海翡翠东方网络信息技术有限公司   
39                高级产品经理(J10274)   14-20k·12薪        广州          广州金鹏集团有限公司   
40                          产品经理           面议    广州-天河区      上海源慧信息科技股份有限公司   
41                          产品经理    8-13k·12薪        广州    深圳合众财富金融投资管理有限公司   
42                      ATS需求分析师           面议        广州              佳都新太科技   
43                    产品经理（临床科研）   15-25k·12薪        广州  健康互联(广州)信息科技股份有限公司   
44                    产品经理-供应链金融   15-30k·12薪    广州-天河区               TCL金融   
45                        直播产品经理   15-25k·15薪    广州-天河区    上海翡翠东方网络信息技术有限公司   
46                 产品经理（全自动运行方向）   10-20k·12薪        广州              佳都新太科技   
47                  产品经理（节能控制方向）   10-20k·12薪        广州              佳都新太科技   
48                 需求分析师（会计运营方向）   12-18k·12薪    广州-天河区     北京公瑾科技有限公司广州分公司   
49                  产品经理（智能运维方向）   10-20k·14薪        广州              佳都新太科技   
50


-----

# 本周目标
* [猎聘PC版](https://www.liepin.com/zhaopin/)
* 上方导航有  公司行业 城市 薪资 的分页选单
* 请练习xpath抽出数据

## Xpath解析HTML

In [15]:
# A-1   单一页面
url = "https://www.liepin.com/zhaopin/?keyword=PRD"
session = HTMLSession()
r = session.get( url )

# 先取特定元素, 精准打击其子后辈
主要元素 = r.html.xpath( \
    '//ul[@class="sojob-list"]/li')

# 预期是一个元素的列表？
#print (主要元素[0].xpath('//div[contains(@class,"sojob-item-main")]'))
#print (主要元素[0].xpath('//div[contains(@class,"job-info")]/h3/a'))
#print (主要元素[3].xpath('//div[contains(@class,"job-info")]/p/a'))
#print (主要元素[3].xpath('//div[contains(@class,"job-info")]/p/span[@class="text-warning"]'))
#print (主要元素[3].xpath('//div[contains(@class,"job-info")]/p/span[@class="edu"]/following-sibling::span'))
#print (主要元素[3].xpath('//div[contains(@class,"job-info")]/p/time/@title'))
#print (主要元素[0].xpath('//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a'))

# 作为xpath字典，键为我要抓的牛肉名称，值为xpath
dict_xpaths={ 
    'text': {
        'edu':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]',
        '经验':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]/following-sibling::span',
        '薪水':    '//div[contains(@class,"job-info")]/p/span[@class="text-warning"]', 
        '时间':    '//div[contains(@class,"job-info")]/p/time/@title', 
        '职称':    '//div[contains(@class,"job-info")]/h3/a', 
        '公司地点': '//div[contains(@class,"job-info")]/p/a',
        '公司名称': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
    },
    'text_content': {
    },
    'href': {
        '链结':    '//div[contains(@class,"job-info")]/h3/a', 
        '公司URL': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
    }
}

def get_e_text_content(_xpath_):
    # 高级列表推导
    暂存结果 = [e.xpath(_xpath_)[0].lxml.text_content() for e in 主要元素]
    return(暂存结果)

def get_e_text(_xpath_):
    # 高级列表推导
    暂存结果 = ["".join([x.strip() if type(x) is str else x.text.strip() for x in e.xpath(_xpath_)]) for e in 主要元素]
    return(暂存结果)

def get_e_href(_xpath_):
    # 高级列表推导
    暂存结果 = [list(e.xpath(_xpath_, first=True).absolute_links)[0] \
               if len(e.xpath(_xpath_, first=True).absolute_links) >= 1  \
               else "" for e in 主要元素]
    return(暂存结果)

# 只对主要元素下进行.xpath取值
数据字典 = dict()

数据字典 = {k:get_e_text_content(v) for k,v in dict_xpaths['text_content'].items()}
数据字典.update({k:get_e_text(v) for k,v in dict_xpaths['text'].items()})
数据字典.update({k:get_e_href(v) for k,v in dict_xpaths['href'].items()})

[len(v) for k,v in 数据字典.items()]

数据 = pd.DataFrame(数据字典)
数据.to_excel("20春_Web数据挖掘_week03_liepin.xlsx", sheet_name="搜查结果")
数据 

edu     经验          薪水           时间                          职称  \
0   大专及以上   2年以上   8-10k·12薪  2020年03月30日                     学术代表-太原   
1   本科及以上   3年以上   8-15k·12薪  2020年03月30日                招聘专员(J10483)   
2   大专及以上   3年以上     13k·12薪  2020年03月30日                招商运营经理（购物中心）   
3   本科及以上   6年以上          面议  2020年03月30日            产品leader— 企业智能方向   
4    学历不限   3年以上    6-9k·12薪  2020年03月30日                        模具钳工   
5    统招本科   5年以上          面议  2020年03月30日             伺服电机驱动控制工程师（硬件）   
6    统招本科   5年以上          面议  2020年03月30日                      工程项目经理   
7    统招本科   经验不限    3-5k·12薪  2020年03月30日                       法务实习生   
8    统招本科   5年以上  15-30k·12薪  2020年03月30日       项目/产品经理（供应链金融&木材贸易方向）   
9   大专及以上   3年以上    6-8k·13薪  2020年03月30日                        核价专员   
10  硕士及以上   1年以上          面议  2020年03月30日  Executive Assistant to CEO   
11  大专及以上   3年以上  13-16k·12薪  2020年03月30日                    跨境电商运营经理   
12  大专及以上   3年以上  12-15k·12薪  2020年03月30日                       品牌培训师   
13  大专及以上   3年以上  13-20k·12薪  2020年03月30日                 京东电商运营经理/店长   
14  本科及以上   3年以上   8-15k·13薪  2020年03月30日                    销售经理（南宁）   
15  本科及以上   2年以上          面议  2020年03月30日              华兴资本-人力资源部-招聘岗   
16  大专及以上  10年以上  20-35k·12薪  2020年03月30日                     物业公司总经理   
17  本科及以上   5年以上  10-15k·13薪  2020年03月30日                    运维经理（博创）   
18  本科及以上   1年以上    7-8k·12薪  2020年03月30日                   实验室技术工程师1   
19  大专及以上   3年以上    5-8k·12薪  2020年03月30日                        资深文案   
20   学历不限  10年以上  20-40k·12薪  2020年03月30日              销售总经理（中高端酒水品牌）   
21  大专及以上   1年以上          面议  2020年03月30日            咨询顾问(保底7-12万,双休)   
22  本科及以上   3年以上   8-11k·12薪  2020年03月30日                     细胞培养技术员   
23  大专及以上   3年以上    6-8k·12薪  2020年03月30日                     全铝家具业务员   
24  大专及以上   3年以上    6-8k·12薪  2020年03月30日                     全铝家具业务员   
25  大专及以上   3年以上    6-8k·12薪  2020年03月30日                     全铝家具业务员   
26  大专及以上   经验不限  10-20k·12薪  2020年03月30日                        渠道经理   
27  本科及以上   经验不限          面议  2020年03月30日           模拟电路设计工程师（电机驱动方向）   
28  大专及以上   1年以上   8-15k·12薪  2020年03月30日             招聘主管/专员(J10156)   
29  大专及以上   5年以上          面议  2020年03月30日                      行业市场经理   
30  本科及以上   3年以上          面议  2020年03月30日                      品牌专业经理   
31  本科及以上   2年以上   8-15k·12薪  2020年03月30日                       橱柜设计师   
32   统招本科   2年以上    6-8k·12薪  2020年03月30日                       党务工作岗   
33     博士   8年以上          面议  2020年03月30日                       首席科学家   
34   统招本科   3年以上          面议  2020年03月30日                    客户关系高级经理   
35  本科及以上   5年以上          面议  2020年03月30日                     工程设备部经理   
36  本科及以上   1年以上          面议  2020年03月30日                     产品开发工程师   
37  大专及以上   5年以上  10-14k·13薪  2020年03月30日                  研发工程师（气雾剂）   
38   统招本科   8年以上          面议  2020年03月30日                 董秘（港股IPO方向）   
39  本科及以上   3年以上    4-6k·12薪  2020年03月30日                      行政管理助理   

       公司地点              公司名称                                           链结  \
0                       博康健基因  https://www.liepin.com/job/1927087201.shtml   
1                       一起住好房  https://www.liepin.com/job/1927086363.shtml   
2        青岛      青岛维客集团股份有限公司  https://www.liepin.com/job/1927085955.shtml   
3   广州-珠江新城              字节跳动  https://www.liepin.com/job/1927085913.shtml   
4        上海        上海新松家电有限公司  https://www.liepin.com/job/1927085613.shtml   
5     上海-七宝              航空电器  https://www.liepin.com/job/1927083551.shtml   
6     厦门-莲前              建发房产  https://www.liepin.com/job/1927082855.shtml   
7    深圳-南山区     深圳市鼎心资本管理有限公司  https://www.liepin.com/job/1927082505.shtml   
8        上海    上海铁林投资控股有限责任公司  https://www.liepin.com/job/1927082147.shtml   
9    佛山-顺德区              飞鱼集团  https://www.liepin.com/job/1927081297.shtml   
10  上海-浦东新区        森浦Sumscope  https://www.liepin.com/job/1927081279.shtml   
11               北京金色三麦商贸有限公司  https://www.liepin.com/job/1927080481.shtml  

In [16]:
# A-2 扩张 公司 ?  

# 先取特定元素, 精准打击其子后辈
主要元素 = r.html.xpath('//div[@data-selector="search-conditions"]')
# 预期是一个元素的列表？
print (主要元素)
print (主要元素[0])
print (主要元素[0].xpath('//dt[@class="search-title"]'))

list_search_title = 主要元素[0].xpath('//dt[@class="search-title"]')
for x in list_search_title:
    print (x.text)
    
list_search_dd = 主要元素[0].xpath('//dt[@class="search-title"]/following-sibling::dd')
for x in list_search_dd:
    print (x)  
    

公司数据选择器链结 = r.html.xpath('//div[@data-selector="search-conditions"]')[0] \
                    .xpath('//dt[@class="search-title"]/following-sibling::dd')[0] \
                    .xpath('//div[contains(@class,"hot-comp-tags")]/a/@href')
               
公司数据选择器链结

# 但我们需要知道这些选择器链结, 对映到什麽数据
公司数据选择器链结 = r.html.xpath('//div[@data-selector="search-conditions"]')[0] \
                    .xpath('//dt[@class="search-title"]/following-sibling::dd')[0] \
                    .xpath('//div[contains(@class,"hot-comp-tags")]/a')
公司数据选择器链结

#[ x.xpath("a/@href")[0] for x in 公司数据选择器链结]
#[ x.xpath("a/text()")[0] for x in 公司数据选择器链结]
公司数据选择器链结 = { x.xpath("a/text()")[0]:x.xpath("a/@href")[0] for x in 公司数据选择器链结}
公司数据选择器链结

[<Element 'div' class=('search-conditions',) data-selector='search-conditions'>]
<Element 'div' class=('search-conditions',) data-selector='search-conditions'>
[<Element 'dt' class=('search-title',)>, <Element 'dt' class=('search-title',)>, <Element 'dt' class=('search-title',)>, <Element 'dt' class=('search-title',)>, <Element 'dt' class=('search-title',)>]
公司：
行业：
城市：
薪资：
更多：
<Element 'dd' class=('comp-list',)>
<Element 'dd' class=('short-dd', 'select-industry') data-param='industries'>
<Element 'dd' data-param='city'>
<Element 'dd' data-param='salary'>
<Element 'dd' class=('dropdown', 'dropdown-time')>
<Element 'dd' class=('dropdown', 'dropdown-jobkind')>
<Element 'dd' class=('dropdown', 'dropdown-compscale')>
<Element 'dd' class=('dropdown', 'dropdown-compkind')>


{'中国500强': '/zhaopin/?init=-1&headckid=51bbfc045663e920&flushckid=1&fromSearchBtn=2&keyword=PRD&compTag=155&ckid=51bbfc045663e920&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=9d755425593cbdf1182f964079a43791&d_curPage=0&d_pageSize=40&d_headId=9d755425593cbdf1182f964079a43791',
 '2018互联网300强': '/zhaopin/?init=-1&headckid=51bbfc045663e920&flushckid=1&fromSearchBtn=2&keyword=PRD&compTag=182&ckid=51bbfc045663e920&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=9d755425593cbdf1182f964079a43791&d_curPage=0&d_pageSize=40&d_headId=9d755425593cbdf1182f964079a43791',
 '制造业500强': '/zhaopin/?init=-1&headckid=51bbfc045663e920&flushckid=1&fromSearchBtn=2&keyword=PRD&compTag=186&ckid=51bbfc045663e920&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=9d755425593cbdf1182f964079a43791&d_curPage=0&d_pageSize=40&d_headId=9d755425593cbdf1182f964079a43791',
 'AI创新成长50强 ': '/zhaopin/?init=-1&headckid=

## 使用urllib3 解析 url 
上面的url应该触动不同的页面查询，但能不能轻松无误的拆分url并进行比较？

### urlparse 模块功能介绍
返回的6个部分，分别是：scheme(机制)丶netloc(网络位置)丶path(路径)丶params(路径段参数)丶query(查询)丶fragment(片段)。

### parse_qs 模块功能介绍
返回多个部分

In [17]:
# B-1 使用 urllib.parse 解析
from urllib.parse import urlparse, parse_qs
[ urlparse(x) for x in 公司数据选择器链结.values()]

[ParseResult(scheme='', netloc='', path='/zhaopin/', params='', query='init=-1&headckid=51bbfc045663e920&flushckid=1&fromSearchBtn=2&keyword=PRD&compTag=155&ckid=51bbfc045663e920&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=9d755425593cbdf1182f964079a43791&d_curPage=0&d_pageSize=40&d_headId=9d755425593cbdf1182f964079a43791', fragment=''),
 ParseResult(scheme='', netloc='', path='/zhaopin/', params='', query='init=-1&headckid=51bbfc045663e920&flushckid=1&fromSearchBtn=2&keyword=PRD&compTag=182&ckid=51bbfc045663e920&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=9d755425593cbdf1182f964079a43791&d_curPage=0&d_pageSize=40&d_headId=9d755425593cbdf1182f964079a43791', fragment=''),
 ParseResult(scheme='', netloc='', path='/zhaopin/', params='', query='init=-1&headckid=51bbfc045663e920&flushckid=1&fromSearchBtn=2&keyword=PRD&compTag=186&ckid=51bbfc045663e920&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom

In [18]:
# B-2 使用 pd.DataFrame进行 unuinque()相异值计量比对 
import pandas as pd
df = pd.DataFrame([ urlparse(x) for x in 公司数据选择器链结.values()])
df.info()
df.nunique()
df.head(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   scheme    6 non-null      object
 1   netloc    6 non-null      object
 2   path      6 non-null      object
 3   params    6 non-null      object
 4   query     6 non-null      object
 5   fragment  6 non-null      object
dtypes: object(6)
memory usage: 416.0+ bytes


scheme netloc       path params  \
0                /zhaopin/          

                                               query fragment  
0  init=-1&headckid=51bbfc045663e920&flushckid=1&...

In [19]:
# B-3 针对query 再解析之 
#df_qs = pd.DataFrame([ parse_qs(x) for x in df['query'] ])
df_qs = pd.DataFrame([{k:v[0] for k,v in parse_qs(x).items()} for x in df['query'] ])
print (df_qs.nunique())
df_qs

init             1
headckid         1
flushckid        1
fromSearchBtn    1
keyword          1
compTag          6
ckid             1
siTag            1
d_sfrom          1
d_ckId           1
d_curPage        1
d_pageSize       1
d_headId         1
dtype: int64


init          headckid flushckid fromSearchBtn keyword compTag  \
0   -1  51bbfc045663e920         1             2     PRD     155   
1   -1  51bbfc045663e920         1             2     PRD     182   
2   -1  51bbfc045663e920         1             2     PRD     186   
3   -1  51bbfc045663e920         1             2     PRD     189   
4   -1  51bbfc045663e920         1             2     PRD     130   
5   -1  51bbfc045663e920         1             2     PRD     156   

               ckid                                          siTag  \
0  51bbfc045663e920  1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw   
1  51bbfc045663e920  1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw   
2  51bbfc045663e920  1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw   
3  51bbfc045663e920  1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw   
4  51bbfc045663e920  1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw   
5  51bbfc045663e920  1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw   

          d_sfrom                            d_ckId d_curPage d_pageSize  \
0  search_unknown  9d755425593cbdf1182f964079a43791         0         40   
1  search_unknown  9d755425593cbdf1182f964079a43791         0         40   
2  search_unknown  9d755425593cbdf1182f964079a43791         0         40   
3  search_unknown  9d755425593cbdf1182f964079a43791         0         40   
4  search_unknown  9d755425593cbdf1182f964079a43791         0         40   
5  search_unknown  9d755425593cbdf1182f964079a43791         0         40   

                           d_headId  
0  9d755425593cbdf1182f964079a43791  
1  9d755425593cbdf1182f964079a43791  
2  9d755425593cbdf1182f964079a43791  
3  9d755425593cbdf1182f964079a43791  
4  9d755425593cbdf1182f964079a43791  
5  9d755425593cbdf1182f964079a43791

### 小结
* comTag 是不同的公司选择器, 数值不样, 对映到不同类型的公司
* keyword 是搜查关键字

In [20]:
# B-4 建构 参数模板 及 字典_compTag
def parse_url_qs_for_compTag (url):
    six_parts = urlparse(url) 
    out = parse_qs(six_parts.query)
    return (out)

# parse_url_qs_for_compTag(list(公司数据选择器链结.values())[0])['compTag']
参数模板 = parse_url_qs_for_compTag(list(公司数据选择器链结.values())[0])
print(参数模板)
# [ parse_url_qs_for_compTag(x)['compTag'] for x in 公司数据选择器链结.values()]
[ parse_url_qs_for_compTag(x)['compTag'][0] for x in 公司数据选择器链结.values()]

字典_compTag = { k:parse_url_qs_for_compTag(v)['compTag'][0] for k,v in 公司数据选择器链结.items()}
print (字典_compTag)


{'init': ['-1'], 'headckid': ['51bbfc045663e920'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'keyword': ['PRD'], 'compTag': ['155'], 'ckid': ['51bbfc045663e920'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['9d755425593cbdf1182f964079a43791'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['9d755425593cbdf1182f964079a43791']}
{'中国500强': '155', '2018互联网300强': '182', '制造业500强': '186', 'AI创新成长50强 ': '189', '独角兽': '130', '上市公司': '156'}


In [21]:
# B-5 建构 参数模板  
def 参数模板生成(compTag , keyword ):
    参数 = 参数模板.copy()
    参数['compTag'] = compTag
    参数['keyword'] = keyword
    return (参数)

参数_compTag_用户体验 = { k:参数模板生成(compTag = [v], keyword = ['用户体验']) for k,v in 字典_compTag.items()}
print(参数_compTag_用户体验)

{'中国500强': {'init': ['-1'], 'headckid': ['51bbfc045663e920'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'keyword': ['用户体验'], 'compTag': ['155'], 'ckid': ['51bbfc045663e920'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['9d755425593cbdf1182f964079a43791'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['9d755425593cbdf1182f964079a43791']}, '2018互联网300强': {'init': ['-1'], 'headckid': ['51bbfc045663e920'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'keyword': ['用户体验'], 'compTag': ['182'], 'ckid': ['51bbfc045663e920'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['9d755425593cbdf1182f964079a43791'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['9d755425593cbdf1182f964079a43791']}, '制造业500强': {'init': ['-1'], 'headckid': ['51bbfc045663e920'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'keyword': ['用户体验'], 'compTag': ['186'], 'ckid': ['51bbfc045663e920'], '

## requests 生成

In [22]:
payload

NameError: name 'payload' is not defined

In [23]:
# C-1   多个页面准备测试1 中国500强
url = "https://www.liepin.com/zhaopin/"
session = HTMLSession()
payload = 参数_compTag_用户体验['中国500强']
r = session.get( url, params = payload)
r.url

'https://www.liepin.com/zhaopin/?init=-1&headckid=51bbfc045663e920&flushckid=1&fromSearchBtn=2&keyword=%E7%94%A8%E6%88%B7%E4%BD%93%E9%AA%8C&compTag=155&ckid=51bbfc045663e920&siTag=1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=9d755425593cbdf1182f964079a43791&d_curPage=0&d_pageSize=40&d_headId=9d755425593cbdf1182f964079a43791'

In [24]:
# C-2  简化 A-1   单一页面爬+解析
session = HTMLSession()

def requests_liepin( url, params):
    r = session.get( url , params = payload)

    # 先取特定元素, 精准打击其子后辈
    主要元素 = r.html.xpath( '//ul[@class="sojob-list"]/li')

    # 作为xpath字典，键为我要抓的牛肉名称，值为xpath
    dict_xpaths={ 
        'text': {
            'edu':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]',
            '经验':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]/following-sibling::span',
            '薪水':    '//div[contains(@class,"job-info")]/p/span[@class="text-warning"]', 
            '时间':    '//div[contains(@class,"job-info")]/p/time/@title', 
            '职称':    '//div[contains(@class,"job-info")]/h3/a', 
            '公司地点': '//div[contains(@class,"job-info")]/p/a',
            '公司名称': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
        },
        'text_content': {
        },
        'href': {
            '链结':    '//div[contains(@class,"job-info")]/h3/a', 
            '公司URL': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
        }
    }

    def get_e_text_content(_xpath_):
        # 高级列表推导
        暂存结果 = [e.xpath(_xpath_)[0].lxml.text_content() for e in 主要元素]
        return(暂存结果)

    def get_e_text(_xpath_):
        # 高级列表推导
        暂存结果 = ["".join([x.strip() if type(x) is str else x.text.strip() for x in e.xpath(_xpath_)]) for e in 主要元素]
        return(暂存结果)

    def get_e_href(_xpath_):
        # 高级列表推导
        暂存结果 = [list(e.xpath(_xpath_, first=True).absolute_links)[0] \
                   if len(e.xpath(_xpath_, first=True).absolute_links) >= 1  \
                   else "" for e in 主要元素]
        return(暂存结果)

    # 只对主要元素下进行.xpath取值
    数据字典 = dict()

    数据字典 = {k:get_e_text_content(v) for k,v in dict_xpaths['text_content'].items()}
    数据字典.update({k:get_e_text(v) for k,v in dict_xpaths['text'].items()})
    数据字典.update({k:get_e_href(v) for k,v in dict_xpaths['href'].items()})

    数据 = pd.DataFrame(数据字典)
    #数据.to_excel("20春_Web数据挖掘_week03_liepin.xlsx", sheet_name="搜查结果")
    return (数据)




In [28]:
# C-3   多个页面
url = "https://www.liepin.com/zhaopin/"

list_df = list()
for k,v in 参数_compTag_用户体验.items():
    payload = v
    df = requests_liepin( url, params = payload)
    df = df.assign (热门公司类型 = k)    
    list_df.append(df)

df_all = pd.concat(list_df)
df_all

edu    经验          薪水           时间                            职称  \
0   本科及以上  3年以上          面议  2020年03月30日               大客户销售经理-北京-网易严选   
1    统招本科  3年以上          面议  2020年03月30日   阿里云智能事业群-数据技术专家(金融行业)-北京/杭州   
2   本科及以上  3年以上          面议  2020年03月27日  钉钉(Dingtalk)-搜索中心-Java开发技术专家   
3   大专及以上  2年以上    6-8k·13薪  2020年03月25日                        员工关系专员   
4    统招本科  3年以上  15-30k·14薪  2020年03月25日               移动生态战略规划部_投资分析师   
..    ...   ...         ...          ...                           ...   
35   统招本科  3年以上  12-25k·12薪  2020年03月26日                         硬件工程师   
36   统招本科  3年以上  15-30k·12薪  2020年03月26日                       控制算法工程师   
37  本科及以上  1年以上    5-8k·12薪  2020年03月26日                   片区人力资源主任/专员   
38  本科及以上  6年以上  15-20k·13薪  2020年03月26日                       法务经理/主任   
39  本科及以上  2年以上  14-20k·12薪  2020年03月26日                          审计专员   

      公司地点               公司名称                                           链结  \
0   北京-五道口               网易集团  https://www.liepin.com/job/1926756751.shtml   
1       杭州               阿里巴巴  https://www.liepin.com/job/1927063431.shtml   
2       杭州               阿里巴巴  https://www.liepin.com/job/1926996383.shtml   
3   廊坊-广阳区     中国国际技术智力合作有限公司  https://www.liepin.com/job/1926938099.shtml   
4       北京              Baidu  https://www.liepin.com/job/1926957535.shtml   
..     ...                ...                                          ...   
35      北京                康斯特  https://www.liepin.com/job/1920146395.shtml   
36  北京-西北旺                康斯特  https://www.liepin.com/job/1920146253.shtml   
37      中山  碧桂园智慧物业服务集团股份有限公司  https://www.liepin.com/job/1919360705.shtml   
38      深圳       中国南玻集团股份有限公司  https://www.liepin.com/job/1926955487.shtml   
39   北京-小关               蓝帆医疗  https://www.liepin.com/job/1923763615.shtml   

                                      公司URL  热门公司类型  
0   https://www.liepin.com/company/5964833/  中国500强  
1   https://www.liepin.com/company/1072424/  中国500强  
2   https://www.liepin.com/company/1072424/  中国500强  
3   https://www.liepin.com/company/1233751/  中国500强  
4    https://www.liepin.com/company/884492/  中国500强  
..                                      ...     ...  
35  https://www.liepin.com/company/7924011/    上市公司  
36  https://www.liepin.com/company/7924011/    上市公司  
37  https://www.liepin.com/company/8694860/    上市公司  
38  https://www.liepin.com/company/9091167/    上市公司  
39  https://www.liepin.com/company/8786211/    上市公司  

[240 rows x 10 columns]

In [29]:
# C-4   输出
df_all.to_excel("20春_Web数据挖掘_week03_liepin_各热门公司类型.xlsx", sheet_name="搜查结果")

In [33]:
# C-5 Pandas  基本能力

print (df_all.nunique())
df_all[['edu']].drop_duplicates()

df_all.groupby(['公司名称','edu']).agg({"职称":"count"}).sort_values(by='职称', ascending=False)

edu         6
经验         10
薪水         64
时间         27
职称        152
公司地点       69
公司名称       56
链结        190
公司URL      56
热门公司类型      6
dtype: int64


职称
公司名称         edu      
天能电池集团股份有限公司 本科及以上  43
华为           统招本科   28
             本科及以上  18
科大讯飞         本科及以上  14
华为           大专及以上   8
...                 ..
智能管家         大专及以上   1
朴新教育         统招本科    1
柳工机械         统招本科    1
江南布衣         大专及以上   1
龙信集团         统招本科    1

[77 rows x 1 columns]

In [34]:
df_all.groupby(['公司名称','edu']).agg({"职称":"count"}).sort_values(by='职称', ascending=False)

职称
公司名称         edu      
天能电池集团股份有限公司 本科及以上  43
华为           统招本科   28
             本科及以上  18
科大讯飞         本科及以上  14
华为           大专及以上   8
...                 ..
智能管家         大专及以上   1
朴新教育         统招本科    1
柳工机械         统招本科    1
江南布衣         大专及以上   1
龙信集团         统招本科    1

[77 rows x 1 columns]

# 本周练习

一样反向工程解析:

## 上方界面的params参数
* 公司：
* 行业：
* 城市：
* 薪资：
## 下方界面的params参数
* 跳转到 N 页确定
## 换  
